<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/OpenCV%EB%A1%9C_Canny_%2B_Hough%EB%A1%9C_%EC%B0%A8%EC%84%A0_%EA%B0%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: OpenCV로 Canny + Hough로 차선 감지

Step 2: ROI 적용 및 영상 처리

Step 3: CNN을 사용한 간단한 edge 분류 (후속 실습)

In [ ]:
# 구글 드라이브 마운트 (선택)
from google.colab import drive
drive.mount('/content/drive')

# OpenCV 설치
!pip install opencv-python
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# yt-dlp 설치
!pip install yt-dlp

# 원하는 유튜브 영상 다운로드
!yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 https://www.youtube.com/shorts/s5ynrR1x9ig


! 업로드하신 영상(mp4 등) 파일은 cv2.imread()로는 읽을 수 없고, **cv2.VideoCapture()**를 사용해서 프레임 단위로 추출해야

In [ ]:
import cv2
from google.colab import files
import matplotlib.pyplot as plt

# 영상 파일 업로드
uploaded = files.upload()

# 파일 이름 가져오기
video_path = list(uploaded.keys())[0]
print("업로드한 영상:", video_path)

# OpenCV로 영상 열기
cap = cv2.VideoCapture(video_path)

# 첫 프레임 읽기
ret, frame = cap.read()
cap.release()

if not ret:
    print("❌ 영상을 읽을 수 없습니다.")
else:
    # 프레임을 출력해보기
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("첫 번째 프레임")
    plt.axis('off')
    plt.show()


In [ ]:
첫 프레임만 차선 인식

In [ ]:
import cv2
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np

# 영상 업로드
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 영상에서 첫 프레임 가져오기
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()

if not ret:
    raise ValueError("영상을 읽을 수 없습니다.")

# 1. 그레이스케일 변환
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 2. 블러 → 에지(Canny)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blur, 50, 150)

# 3. ROI 설정 (영상 하단 절반)
height, width = edges.shape
mask = np.zeros_like(edges)
polygon = np.array([[
    (0, height),
    (width, height),
    (width, int(height * 0.6)),
    (0, int(height * 0.6))
]])
cv2.fillPoly(mask, polygon, 255)
roi = cv2.bitwise_and(edges, mask)

# 4. Hough Transform으로 직선 검출
lines = cv2.HoughLinesP(roi, 2, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=50)

# 5. 원본 프레임에 선 그리기
line_image = frame.copy()
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

# 6. 결과 출력
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("원본 프레임")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
plt.title("차선 인식 결과")
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab import files

# 영상 업로드
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 원본 영상 읽기
cap = cv2.VideoCapture(video_path)

# 영상 정보 가져오기
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# 결과 저장용 비디오 라이터 설정
out = cv2.VideoWriter('lane_detected_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. 그레이 변환 & 블러 & 에지
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # 2. ROI 마스크
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (0, height),
        (width, height),
        (width, int(height * 0.6)),
        (0, int(height * 0.6))
    ]])
    cv2.fillPoly(mask, polygon, 255)
    roi = cv2.bitwise_and(edges, mask)

    # 3. 허프 변환
    lines = cv2.HoughLinesP(roi, 2, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=50)

    # 4. 차선 그리기
    line_image = frame.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # 5. 프레임 저장
    out.write(line_image)

cap.release()
out.release()
print("✅ 완료: lane_detected_output.mp4 저장됨!")


# Ultra-Fast Lane Detection - YouTube 영상 처리

UltraFastLaneDetection는 pip로 설치하는 패키지가 아니에요! 😅
🔍 문제:

UltraFastLaneDetection은  클래스

In [ ]:
# 깔끔한 Ultra-Fast Lane Detection
# Google Colab + Google Drive 전용

# 1. 라이브러리 설치 및 임포트
print("📦 라이브러리 설치 중...")
!pip install torch torchvision opencv-python-headless

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

print("🚗 Ultra-Fast Lane Detection 시작!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"💻 GPU 사용 가능: {torch.cuda.is_available()}")

# 2. 간단한 Lane Detection 클래스
class UltraFastLaneDetection:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        print(f"🔧 디바이스: {self.device}")

    def create_model(self):
        """간단한 차선 검출 모델 생성"""
        print("🧠 차선 검출 모델 생성 중...")

        class SimpleLaneNet(torch.nn.Module):
            def __init__(self):
                super(SimpleLaneNet, self).__init__()
                self.features = torch.nn.Sequential(
                    torch.nn.Conv2d(3, 32, 3, padding=1),
                    torch.nn.ReLU(),
                    torch.nn.MaxPool2d(2),
                    torch.nn.Conv2d(32, 64, 3, padding=1),
                    torch.nn.ReLU(),
                    torch.nn.MaxPool2d(2),
                    torch.nn.Conv2d(64, 1, 1),  # 차선 마스크 생성
                    torch.nn.Sigmoid()
                )

            def forward(self, x):
                return self.features(x)

        self.model = SimpleLaneNet().to(self.device)
        self.model.eval()
        print("✅ 모델 생성 완료")

    def detect_lanes_traditional(self, image):
        """전통적 방법으로 차선 검출 (빠른 결과용)"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 50, 150)

        # 관심 영역 설정
        h, w = edges.shape
        mask = np.zeros_like(edges)
        polygon = np.array([[
            (0, h), (w//2-50, h//2+50), (w//2+50, h//2+50), (w, h)
        ]], np.int32)
        cv2.fillPoly(mask, polygon, 255)
        masked = cv2.bitwise_and(edges, mask)

        # 허프 변환
        lines = cv2.HoughLinesP(masked, 1, np.pi/180, 50, minLineLength=100, maxLineGap=50)

        # 차선 그리기
        result = image.copy()
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                # 차선 기울기 필터링
                if abs(x2 - x1) > 20:  # 너무 수직인 선 제외
                    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 3)

        return result

    def process_frame(self, frame):
        """단일 프레임 처리"""
        return self.detect_lanes_traditional(frame)

# 3. 영상 처리 함수
def process_video_simple(video_path, output_path='lane_result.mp4'):
    """간단한 영상 처리"""
    print(f"🎬 영상 처리 시작: {video_path}")

    # 파일 존재 확인
    if not os.path.exists(video_path):
        print(f"❌ 파일이 존재하지 않습니다: {video_path}")
        return None, []

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다")
        return None, []

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"📊 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 영상 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 차선 검출기 초기화
    detector = UltraFastLaneDetection()
    detector.create_model()

    frame_count = 0
    sample_frames = []

    print("🚀 프레임별 처리 시작...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 차선 검출
        result_frame = detector.process_frame(frame)

        # 정보 텍스트 추가
        cv2.putText(result_frame, "Lane Detection",
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(result_frame, f"Frame: {frame_count}/{total_frames}",
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(result_frame)

        # 샘플 프레임 저장 (4개)
        if frame_count % max(1, total_frames // 4) == 0:
            sample_frames.append(result_frame.copy())

        frame_count += 1

        # 진행률 표시
        if frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 진행률: {progress:.1f}%")

    cap.release()
    out.release()

    print(f"✅ 영상 처리 완료: {output_path}")
    print(f"📊 처리된 프레임: {frame_count}개")

    return output_path, sample_frames

# 4. 결과 시각화
def show_results(sample_frames):
    """결과 시각화"""
    if not sample_frames:
        print("⚠️ 표시할 샘플이 없습니다")
        return

    num_samples = min(4, len(sample_frames))

    if num_samples == 1:
        plt.figure(figsize=(10, 6))
        frame_rgb = cv2.cvtColor(sample_frames[0], cv2.COLOR_BGR2RGB)
        plt.imshow(frame_rgb)
        plt.title('Lane Detection 결과')
        plt.axis('off')
    else:
        rows = 2 if num_samples > 2 else 1
        cols = 2 if num_samples > 1 else 1

        fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
        if num_samples == 1:
            axes = [axes]
        elif rows == 1:
            axes = [axes] if cols == 1 else axes
        else:
            axes = axes.flatten()

        for i in range(num_samples):
            frame_rgb = cv2.cvtColor(sample_frames[i], cv2.COLOR_BGR2RGB)
            axes[i].imshow(frame_rgb)
            axes[i].set_title(f'샘플 {i+1}')
            axes[i].axis('off')

        # 빈 subplot 숨기기
        for i in range(num_samples, len(axes)):
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# 5. 메인 실행 함수
def run_lane_detection_drive(video_filename):
    """Google Drive 영상으로 차선 검출 실행"""

    # Drive 마운트
    print("📁 Google Drive 마운트...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print("✅ Drive 마운트 완료")
    except:
        print("⚠️ Drive 마운트 실패, 이미 마운트되어 있을 수 있습니다")

    # 영상 파일 경로 설정
    video_path = f"/content/drive/MyDrive/{video_filename}"

    print(f"🔍 파일 찾기: {video_path}")

    # 파일 존재 확인
    if not os.path.exists(video_path):
        print(f"❌ 파일을 찾을 수 없습니다: {video_filename}")
        print("📁 Drive 파일 목록:")
        try:
            files = os.listdir("/content/drive/MyDrive/")
            video_files = [f for f in files if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
            for f in video_files:
                print(f"   📹 {f}")
        except:
            print("   Drive 폴더에 접근할 수 없습니다")
        return None, None

    # 영상 처리
    output_path, sample_frames = process_video_simple(video_path)

    if output_path:
        # 결과 시각화
        show_results(sample_frames)

        # 파일 크기 정보
        file_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"📊 결과 파일 크기: {file_size:.1f} MB")
        print(f"📁 결과 파일: {output_path}")

        return output_path, sample_frames
    else:
        print("❌ 영상 처리 실패")
        return None, None

# 6. 사용법 안내
print("\n🎯 사용법:")
print("="*50)
print("1. Google Drive에 영상 파일 업로드")
print("2. 아래 함수 실행:")
print("   result_path, samples = run_lane_detection_drive('your_video.mp4')")
print("\n📺 예제:")
print("   run_lane_detection_drive('seoul_road.mp4')")
print("="*50)

# 7. 빠른 실행 함수
def quick_run():
    """빠른 실행 (파일명 자동 감지)"""
    print("🔍 Drive에서 영상 파일 자동 검색...")

    try:
        drive.mount('/content/drive', force_remount=False)
        files = os.listdir("/content/drive/MyDrive/")
        video_files = [f for f in files if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

        if video_files:
            print(f"🎬 발견된 영상 파일들:")
            for i, f in enumerate(video_files):
                print(f"   {i+1}. {f}")

            # 첫 번째 파일로 실행
            first_video = video_files[0]
            print(f"\n🚀 첫 번째 파일로 실행: {first_video}")
            return run_lane_detection_drive(first_video)
        else:
            print("❌ Drive에 영상 파일이 없습니다")
            print("💡 .mp4, .avi, .mov, .mkv 파일을 업로드해주세요")
            return None, None

    except Exception as e:
        print(f"❌ 오류: {e}")
        return None, None

print("\n🚀 빠른 시작: quick_run() 실행!")

In [ ]:
# 결과 파일 존재 확인
import os
result_file = "/content/lane_result.mp4"
if os.path.exists(result_file):
    print(f"✅ 결과 파일 존재: {result_file}")
    file_size = os.path.getsize(result_file) / (1024*1024)
    print(f"📊 파일 크기: {file_size:.1f} MB")
else:
    print("❌ 결과 파일이 없습니다")

# 모든 mp4 파일 찾기
mp4_files = [f for f in os.listdir("/content/") if f.endswith('.mp4')]
print(f"🎬 발견된 영상 파일들: {mp4_files}")

위의 것의 결과를 못찾음

아래는 영상 다운로드 하는 코드

In [ ]:
print("📦 라이브러리 설치 중...")
!pip install torch torchvision opencv-python-headless yt-dlp

In [ ]:
print("📦 라이브러리 설치 중...")
!pip install torch torchvision opencv-python-headless yt-dlp
run_youtube_lane_detection("https://www.youtube.com/shorts/yr6V8PX_yGwfmf")

In [ ]:
# https://www.youtube.com/shorts/yr6V8PX_yGw

In [ ]:
# 완전 간단한 차선검출
# 한 번에 실행!

print("🚀 간단한 차선검출 시작!")

# 1. 라이브러리 설치
!pip install yt-dlp opencv-python-headless

import cv2
import numpy as np
import matplotlib.pyplot as plt
import yt_dlp
import os

# 2. YouTube 다운로드
def download_video():
    url = "https://www.youtube.com/shorts/yr6V8PX_yGwfmf"
    output = "/content/2.mp4"

    print(f"📥 다운로드 시작: {url}")

    # 기존 파일 삭제
    if os.path.exists(output):
        os.remove(output)

    try:
        ydl_opts = {
            'outtmpl': output,
            'format': 'best[height<=480]'
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        if os.path.exists(output):
            print(f"✅ 다운로드 완료: {output}")
            return output
        else:
            print("❌ 다운로드 실패")
            return None
    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

# 3. 차선 검출
def detect_lanes(image):
    # 그레이스케일
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 블러
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # 엣지 검출
    edges = cv2.Canny(blur, 50, 150)

    # 관심영역
    h, w = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (0, h), (w//2-50, h//2+50), (w//2+50, h//2+50), (w, h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked = cv2.bitwise_and(edges, mask)

    # 허프 변환
    lines = cv2.HoughLinesP(masked, 1, np.pi/180, 50, minLineLength=100, maxLineGap=50)

    # 차선 그리기
    result = image.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x2 - x1) > 20:  # 수직선 제외
                cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 4)

    return result

# 4. 영상 처리
def process_video(input_path):
    output_path = "/content/lane_result.mp4"

    print(f"🎬 영상 처리 시작: {input_path}")

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다")
        return None

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"📊 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    sample_frames = []

    print("🚀 처리 중...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 차선 검출
        result_frame = detect_lanes(frame)

        # 텍스트 추가
        cv2.putText(result_frame, "Lane Detection",
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(result_frame, f"{frame_count+1}/{total_frames}",
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(result_frame)

        # 샘플 저장
        if frame_count % max(1, total_frames // 4) == 0:
            sample_frames.append(result_frame)

        frame_count += 1

        # 진행률
        if frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 {progress:.0f}%")

    cap.release()
    out.release()

    print(f"✅ 완료! 결과: {output_path}")

    # 샘플 보기
    if sample_frames:
        plt.figure(figsize=(15, 10))
        for i, frame in enumerate(sample_frames[:4]):
            plt.subplot(2, 2, i+1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.imshow(frame_rgb)
            plt.title(f'샘플 {i+1}')
            plt.axis('off')
        plt.tight_layout()
        plt.show()

    return output_path

# 5. 전체 실행
def run_all():
    print("🎯 YouTube 차선검출 시작!")

    # 1. 다운로드
    video_path = download_video()
    if not video_path:
        print("❌ 다운로드 실패")
        return None

    # 2. 처리
    result_path = process_video(video_path)
    if result_path:
        print(f"🎉 성공! 결과 파일: {result_path}")
        print("📁 왼쪽 파일탭에서 'lane_result.mp4' 다운로드 가능")
        return result_path
    else:
        print("❌ 처리 실패")
        return None

# 실행!
print("🚀 사용법: run_all() 실행하세요!")
print("="*40)

In [ ]:
# 새 셀에서 실행
def fix_download():
    url = "https://www.youtube.com/shorts/yr6V8PX_yGwfmf"
    output = "/content/2.mp4"

    if os.path.exists(output):
        os.remove(output)

    # 더 유연한 옵션들
    ydl_opts = {
        'outtmpl': output,
        'format': 'best/worst',  # 사용 가능한 아무 포맷
        'no_warnings': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        if os.path.exists(output):
            print(f"✅ 다운로드 성공!")
            return process_video(output)
        else:
            print("❌ 파일 생성 실패")
    except Exception as e:
        print(f"❌ 오류: {e}")

fix_download()

In [ ]:
# 개선된 차선검출 - 순차적 필터링
# 버스, 자동차 선 제거하고 진짜 차선만 감지!

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

print("🔧 개선된 차선검출 시작!")

# 1. 색상 기반 필터링 (흰색/노란색 차선만)
def detect_lane_colors(image):
    """흰색과 노란색 차선만 감지"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 흰색 차선 마스크 (밝기 높은 영역)
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([180, 30, 255])
    white_mask = cv2.inRange(hsv, white_lower, white_upper)

    # 노란색 차선 마스크
    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)

    # 두 마스크 결합
    lane_mask = cv2.bitwise_or(white_mask, yellow_mask)

    return lane_mask

# 2. 위치 기반 필터링 (도로 하단부만)
def filter_by_position(lines, image_shape):
    """도로 하단 영역의 선만 필터링"""
    if lines is None:
        return []

    h, w = image_shape[:2]
    filtered_lines = []

    for line in lines:
        x1, y1, x2, y2 = line[0]

        # 조건 1: 하단 40% 영역에 있는 선만
        if y1 > h * 0.6 and y2 > h * 0.6:
            # 조건 2: 화면 중앙 근처의 선만 (좌우 극단 제외)
            center_x = w // 2
            line_center_x = (x1 + x2) // 2

            if abs(line_center_x - center_x) < w * 0.4:  # 중앙에서 40% 범위 내
                filtered_lines.append(line)

    return filtered_lines

# 3. 기울기 필터링 (차선 각도만)
def filter_by_slope(lines):
    """적절한 기울기의 선만 필터링"""
    if not lines:
        return []

    filtered = []
    for line in lines:
        x1, y1, x2, y2 = line[0]

        if abs(x2 - x1) > 20:  # 너무 수직인 선 제외
            slope = abs((y2 - y1) / (x2 - x1)) if (x2 - x1) != 0 else 999

            # 차선다운 기울기만 (0.3 ~ 3.0)
            if 0.3 < slope < 3.0:
                filtered.append(line)

    return filtered

# 4. 개선된 차선 검출 함수
def detect_lanes_improved(image):
    """개선된 차선 검출 - 3단계 필터링"""

    # 단계 1: 색상 기반 필터링
    lane_mask = detect_lane_colors(image)

    # 단계 2: 에지 검출 (색상 필터링된 영역에서)
    blur = cv2.GaussianBlur(lane_mask, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # 단계 3: 관심 영역 마스크 (더 정교하게)
    h, w = edges.shape
    mask = np.zeros_like(edges)

    # 도로 영역만 (사다리꼴 모양)
    polygon = np.array([[
        (w * 0.1, h),           # 왼쪽 하단
        (w * 0.4, h * 0.6),     # 왼쪽 상단
        (w * 0.6, h * 0.6),     # 오른쪽 상단
        (w * 0.9, h)            # 오른쪽 하단
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # 단계 4: 허프 변환 (더 엄격한 조건)
    lines = cv2.HoughLinesP(
        masked_edges,
        rho=1,
        theta=np.pi/180,
        threshold=30,        # 더 높은 임계값
        minLineLength=50,    # 더 짧은 최소 길이
        maxLineGap=30        # 더 작은 간격
    )

    # 단계 5: 위치 기반 필터링
    filtered_lines_1 = filter_by_position(lines, image.shape)

    # 단계 6: 기울기 기반 필터링
    filtered_lines_2 = filter_by_slope(filtered_lines_1)

    # 결과 이미지 생성
    result = image.copy()

    if filtered_lines_2:
        for line in filtered_lines_2:
            x1, y1, x2, y2 = line[0]
            cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 4)

    # 디버그 정보 표시
    total_lines = len(lines) if lines is not None else 0
    filtered_lines_count = len(filtered_lines_2)

    cv2.putText(result, f"Total lines: {total_lines}",
               (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(result, f"Filtered: {filtered_lines_count}",
               (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return result, lane_mask

# 5. 개선된 영상 처리 함수
def process_video_improved(input_path):
    """개선된 알고리즘으로 영상 처리"""
    output_path = "/content/improved_lane_result.mp4"

    print(f"🎬 개선된 차선검출 시작: {input_path}")

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다")
        return None

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"📊 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    sample_frames = []
    sample_masks = []

    print("🚀 개선된 알고리즘으로 처리 중...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 개선된 차선 검출
        result_frame, lane_mask = detect_lanes_improved(frame)

        # 텍스트 추가
        cv2.putText(result_frame, "Improved Lane Detection",
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(result_frame, f"Frame: {frame_count+1}/{total_frames}",
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(result_frame)

        # 샘플 저장 (결과 + 마스크)
        if frame_count % max(1, total_frames // 4) == 0:
            sample_frames.append(result_frame)
            sample_masks.append(lane_mask)

        frame_count += 1

        # 진행률
        if frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 {progress:.0f}%")

    cap.release()
    out.release()

    print(f"✅ 개선된 처리 완료: {output_path}")

    # 결과 비교 시각화
    if sample_frames and sample_masks:
        fig, axes = plt.subplots(3, 4, figsize=(16, 12))

        for i in range(min(4, len(sample_frames))):
            # 원본 결과
            frame_rgb = cv2.cvtColor(sample_frames[i], cv2.COLOR_BGR2RGB)
            axes[0, i].imshow(frame_rgb)
            axes[0, i].set_title(f'개선된 결과 {i+1}')
            axes[0, i].axis('off')

            # 색상 마스크
            axes[1, i].imshow(sample_masks[i], cmap='gray')
            axes[1, i].set_title(f'색상 필터 {i+1}')
            axes[1, i].axis('off')

            # 원본과 비교용 (기존 방식)
            original_result = detect_lanes_original(sample_frames[i])
            if original_result is not None:
                original_rgb = cv2.cvtColor(original_result, cv2.COLOR_BGR2RGB)
                axes[2, i].imshow(original_rgb)
                axes[2, i].set_title(f'기존 방식 {i+1}')
                axes[2, i].axis('off')

        plt.tight_layout()
        plt.show()

    return output_path

# 6. 기존 방식 (비교용)
def detect_lanes_original(image):
    """기존 방식 (비교용)"""
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 50, 150)

        h, w = edges.shape
        mask = np.zeros_like(edges)
        polygon = np.array([[
            (0, h), (w//2-50, h//2+50), (w//2+50, h//2+50), (w, h)
        ]], np.int32)
        cv2.fillPoly(mask, polygon, 255)
        masked = cv2.bitwise_and(edges, mask)

        lines = cv2.HoughLinesP(masked, 1, np.pi/180, 50, minLineLength=100, maxLineGap=50)

        result = image.copy()
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                if abs(x2 - x1) > 20:
                    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 4)

        return result
    except:
        return None

# 7. 실행 함수
def run_improved():
    """개선된 차선검출 실행"""
    print("🔧 개선된 차선검출 시작!")

    # 방금 다운로드한 2.mp4 파일 사용
    video_file = "/content/2.mp4"

    if os.path.exists(video_file):
        print(f"✅ 파일 확인: {video_file}")
        result_path = process_video_improved(video_file)
        if result_path:
            print(f"🎉 개선된 결과: {result_path}")
            print("📊 기존 방식과 비교해보세요!")
            return result_path
    else:
        print("❌ /content/2.mp4 파일을 찾을 수 없습니다")
        print("💡 먼저 fix_download() 실행해서 영상을 다운로드하세요")
        return None

print("🚀 사용법: run_improved() 실행!")
print("="*50)
print("💡 개선사항:")
print("   1. 색상 필터링 (흰색/노란색만)")
print("   2. 위치 필터링 (도로 하단부만)")
print("   3. 기울기 필터링 (적절한 각도만)")
print("   4. 기존 방식과 비교 시각화")

In [ ]:
# 위 코드 전체 실행 후
run_improved()

In [ ]:
# 색상 필터 테스트
def test_color_filter():
    cap = cv2.VideoCapture("/content/2.mp4")
    ret, frame = cap.read()
    if ret:
        lane_mask = detect_lane_colors(frame)

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title('원본')

        plt.subplot(1, 3, 2)
        plt.imshow(lane_mask, cmap='gray')
        plt.title('색상 필터 결과')

        # HSV 확인
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        plt.subplot(1, 3, 3)
        plt.imshow(hsv[:,:,2], cmap='gray')
        plt.title('밝기 채널')

        plt.show()
    cap.release()

test_color_filter()

위의 결과 분석
색상 필터 결과를 보니 문제를 찾았어요!
🔍 분석:

원본: 도로에 흰색 차선과 횡단보도가 보임
색상 필터: 일부 영역은 감지되지만 너무 적음
밝기 채널: 전체적으로 어두워서 흰색 기준이 너무 높음

🔧 문제와 해결책:
1️⃣ 색상 필터 기준이 너무 엄격
# 현재: white_lower = [0, 0, 200] ← 너무 밝아야 함
# 개선: white_lower = [0, 0, 120] ← 더 관대하게

2️⃣ 즉시 수정 버전

In [ ]:
def detect_lanes_simple_fixed(image):
    """간단하고 효과적인 차선 검출"""
    # 그레이스케일로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 밝은 영역 강조 (차선 부각)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # 블러 + 엣지
    blur = cv2.GaussianBlur(thresh, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # 관심영역 (도로 하단부)
    h, w = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (w*0.1, h), (w*0.45, h*0.6), (w*0.55, h*0.6), (w*0.9, h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked = cv2.bitwise_and(edges, mask)

    # 허프 변환
    lines = cv2.HoughLinesP(masked, 1, np.pi/180, 40, minLineLength=50, maxLineGap=30)

    # 결과 그리기
    result = image.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # 기울기 필터링
            if abs(x2 - x1) > 20:
                slope = abs((y2 - y1) / (x2 - x1)) if (x2 - x1) != 0 else 999
                if 0.3 < slope < 3.0:
                    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 4)

    return result

# 빠른 테스트
cap = cv2.VideoCapture("/content/2.mp4")
ret, frame = cap.read()
if ret:
    fixed_result = detect_lanes_simple_fixed(frame)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('원본')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(fixed_result, cv2.COLOR_BGR2RGB))
    plt.title('수정된 차선검출')
    plt.axis('off')

    plt.show()
cap.release()

개선된 알고리즘으로 전체를 적용

In [ ]:
# 전체 영상에 적용
def process_video_fixed(input_path):
    """수정된 알고리즘으로 전체 영상 처리"""
    output_path = "/content/fixed_lane_result.mp4"

    print(f"🎬 수정된 알고리즘으로 영상 처리: {input_path}")

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다")
        return None

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"📊 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    sample_frames = []

    print("🚀 수정된 알고리즘으로 처리 중...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 수정된 차선 검출
        result_frame = detect_lanes_simple_fixed(frame)

        # 텍스트 추가
        cv2.putText(result_frame, "Fixed Lane Detection",
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(result_frame, f"Frame: {frame_count+1}/{total_frames}",
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(result_frame)

        # 샘플 저장
        if frame_count % max(1, total_frames // 4) == 0:
            sample_frames.append(result_frame)

        frame_count += 1

        # 진행률
        if frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 {progress:.0f}%")

    cap.release()
    out.release()

    print(f"✅ 수정된 처리 완료: {output_path}")

    # 샘플 보기
    if sample_frames:
        plt.figure(figsize=(15, 10))
        for i, frame in enumerate(sample_frames[:4]):
            plt.subplot(2, 2, i+1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.imshow(frame_rgb)
            plt.title(f'수정된 결과 {i+1}')
            plt.axis('off')
        plt.tight_layout()
        plt.show()

    return output_path

# 실행
result_path = process_video_fixed("/content/2.mp4")
if result_path:
    print(f"🎉 완료! 결과: {result_path}")

🚗 원인 분석:

횡단보도: 흰색이 뚜렷해서 임계값 150으로 감지됨
도로 차선: 회색 아스팔트 위의 흰 선이라 대비가 약함

🔧 해결책 - 적응형 임계값:

In [ ]:
def detect_lanes_adaptive(image):
    """적응형 임계값으로 차선 검출"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 1. 적응형 임계값 (지역별로 다른 기준)
    adaptive = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )

    # 2. 일반 임계값도 함께 사용
    _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)  # 기준 낮춤

    # 3. 두 결과 결합
    combined = cv2.bitwise_or(adaptive, thresh)

    # 4. 형태학적 연산으로 노이즈 제거
    kernel = np.ones((3,3), np.uint8)
    cleaned = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel)

    # 5. 블러 + 엣지
    blur = cv2.GaussianBlur(cleaned, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # 6. 관심영역
    h, w = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (w*0.1, h), (w*0.45, h*0.6), (w*0.55, h*0.6), (w*0.9, h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked = cv2.bitwise_and(edges, mask)

    # 7. 허프 변환 (더 민감하게)
    lines = cv2.HoughLinesP(
        masked, 1, np.pi/180,
        threshold=25,      # 더 낮은 임계값
        minLineLength=30,  # 더 짧은 선도 감지
        maxLineGap=20      # 더 작은 간격
    )

    # 8. 결과 그리기
    result = image.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x2 - x1) > 15:  # 수직선 제외
                slope = abs((y2 - y1) / (x2 - x1)) if (x2 - x1) != 0 else 999
                if 0.2 < slope < 5.0:  # 더 넓은 기울기 범위
                    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 4)

    return result, combined

# 테스트
cap = cv2.VideoCapture("/content/2.mp4")
ret, frame = cap.read()
if ret:
    adaptive_result, threshold_result = detect_lanes_adaptive(frame)

    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('원본')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(threshold_result, cmap='gray')
    plt.title('적응형 임계값')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(adaptive_result, cv2.COLOR_BGR2RGB))
    plt.title('개선된 차선검출')
    plt.axis('off')

    plt.show()
cap.release()